In [1]:
import numpy as np

In [47]:
equals = np.array([0,1,0], dtype="bool")
overlaps = np.array([1,1,1], dtype="bool")
included_in = np.array([0,1,1], dtype="bool")
disjoint = np.array([1,0,1], dtype="bool")
includes = np.array([1,1,0], dtype="bool")

In [48]:
sym_dict = {
    "=": equals,
    "o": overlaps,
    "<": included_in,
    ">": includes,
    "!": disjoint
}
all_rels = set(sym_dict.keys())

In [49]:
idx_rel_map = {0: ("in", "out"),
               1: ("in", "in"),
               2: ("out", "in")}

In [50]:
def rec_bitwise_and(fs):
    if len(fs) == 1:
        return fs[0]
    if len(fs) == 2:
        return np.bitwise_and(fs[0], fs[1])
    mid = int(np.ceil(len(fs)/2))
    return np.bitwise_and(rec_bitwise_and(fs[:mid]), rec_bitwise_and(fs[mid:]))

In [51]:
def rec_bitwise_and_not(fs):
    f_not = list(map(np.invert, fs))
    return rec_bitwise_and(f_not)

In [52]:
def intersection(fs):
    t1 = rec_bitwise_and(fs)
    t2 = rec_bitwise_and_not(fs)
    return np.bitwise_or(t1, t2)

In [53]:
def not_filter_helper(r_var, rel1, rel2, n1, n2, sign):
    return "#count {{{0} : vrs({0}), {1}({3}, {0}), {2}({4}, {0})}} {5} 0".format(r_var, rel1, rel2, n1, n2, sign)

In [54]:
def not_filter(n1, n2, rel):
    ts = []
    for i in range(len(rel)):
        t = not_filter_helper(chr(ord('A')+i), idx_rel_map[i][0], idx_rel_map[i][1], n1, n2, ">" if rel[i] else "=")
        ts.append(t)
    return ":- {}.".format(",\n   ".join(ts))

In [55]:
def ir_helper(rel1: str, rel2: str, n1: str, n2: str, idx: int, rel_var="X"):
    return "ir({0}, r{1}) :- {2}({3}, {0}), {4}({5}, {0}).".format(rel_var, idx, rel1, n1, rel2, n2)

In [56]:
def vr_ir_helper(rel1: str, rel2: str, n1: str, n2: str, idx: int, rel_var="X"):
    return "vr({}, r{}) ; ".format(rel_var, idx) + ir_helper(rel1, rel2, n1, n2, idx, rel_var)

In [59]:
def gen_rules(n1, n2, rels):
    """
    Generates the rules that must be encoded in clingo to represent the
    list of possible relations (rels) between given nodes n1 and n2
    """
    global rule_count
    rule_count += 1
    rules = []
    not_rels = list(all_rels - set(rels))
    rels = list(map(lambda x: sym_dict[x], rels))
    not_rels = list(map(lambda x: sym_dict[x], not_rels))
    for not_rel in not_rels:
        rules.append(not_filter(n1, n2, not_rel))
    if len(rels) > 1:
        intersect = intersection(rels)
        for i in range(len(intersect)):
            if intersect[i] == 1 and rels[0][i] == 0:
                rules.append(ir_helper(idx_rel_map[i][0], idx_rel_map[i][1], n1, n2, rule_count))
            elif intersect[i] == 1 and rels[0][i] == 1:
                rules.append(":- {}.".format(not_filter_helper("X", idx_rel_map[i][0], idx_rel_map[i][1], n1, n2, "="), "."))
            elif intersect[i] == 0:
                rules.append(vr_ir_helper(idx_rel_map[i][0], idx_rel_map[i][1], n1, n2, rule_count))
    else:
        for i in range(len(rels[0])):
            if rels[0][i] == 0:
                rules.append(ir_helper(idx_rel_map[i][0], idx_rel_map[i][1], n1, n2, rule_count))
            elif rels[0][i] == 1:
                rules.append(":- {}.".format(not_filter_helper("X", idx_rel_map[i][0], idx_rel_map[i][1], n1, n2, "="), "."))
    return rules

In [57]:
rule_count = 0

In [58]:
rule_count = 21

In [60]:
print("\n".join(gen_rules("1_A", "2_A", ["<", "="])))

:- #count {A : vrs(A), in(1_A, A), out(2_A, A)} > 0,
   #count {B : vrs(B), in(1_A, B), in(2_A, B)} > 0,
   #count {C : vrs(C), out(1_A, C), in(2_A, C)} > 0.
:- #count {A : vrs(A), in(1_A, A), out(2_A, A)} > 0,
   #count {B : vrs(B), in(1_A, B), in(2_A, B)} = 0,
   #count {C : vrs(C), out(1_A, C), in(2_A, C)} > 0.
:- #count {A : vrs(A), in(1_A, A), out(2_A, A)} > 0,
   #count {B : vrs(B), in(1_A, B), in(2_A, B)} > 0,
   #count {C : vrs(C), out(1_A, C), in(2_A, C)} = 0.
ir(X, r22) :- in(1_A, X), out(2_A, X).
:- #count {X : vrs(X), in(1_A, X), in(2_A, X)} = 0.
vr(X, r22) ; ir(X, r22) :- out(1_A, X), in(2_A, X).


In [61]:
print("\n".join(gen_rules("1_B", "2_B", ["="])))

:- #count {A : vrs(A), in(1_B, A), out(2_B, A)} = 0,
   #count {B : vrs(B), in(1_B, B), in(2_B, B)} > 0,
   #count {C : vrs(C), out(1_B, C), in(2_B, C)} > 0.
:- #count {A : vrs(A), in(1_B, A), out(2_B, A)} > 0,
   #count {B : vrs(B), in(1_B, B), in(2_B, B)} > 0,
   #count {C : vrs(C), out(1_B, C), in(2_B, C)} > 0.
:- #count {A : vrs(A), in(1_B, A), out(2_B, A)} > 0,
   #count {B : vrs(B), in(1_B, B), in(2_B, B)} = 0,
   #count {C : vrs(C), out(1_B, C), in(2_B, C)} > 0.
:- #count {A : vrs(A), in(1_B, A), out(2_B, A)} > 0,
   #count {B : vrs(B), in(1_B, B), in(2_B, B)} > 0,
   #count {C : vrs(C), out(1_B, C), in(2_B, C)} = 0.
ir(X, r23) :- in(1_B, X), out(2_B, X).
:- #count {X : vrs(X), in(1_B, X), in(2_B, X)} = 0.
ir(X, r23) :- out(1_B, X), in(2_B, X).


In [62]:
print("\n".join(gen_rules("1_C", "2_F", ["<", "o"])))

:- #count {A : vrs(A), in(1_C, A), out(2_F, A)} = 0,
   #count {B : vrs(B), in(1_C, B), in(2_F, B)} > 0,
   #count {C : vrs(C), out(1_C, C), in(2_F, C)} = 0.
:- #count {A : vrs(A), in(1_C, A), out(2_F, A)} > 0,
   #count {B : vrs(B), in(1_C, B), in(2_F, B)} = 0,
   #count {C : vrs(C), out(1_C, C), in(2_F, C)} > 0.
:- #count {A : vrs(A), in(1_C, A), out(2_F, A)} > 0,
   #count {B : vrs(B), in(1_C, B), in(2_F, B)} > 0,
   #count {C : vrs(C), out(1_C, C), in(2_F, C)} = 0.
vr(X, r24) ; ir(X, r24) :- in(1_C, X), out(2_F, X).
:- #count {X : vrs(X), in(1_C, X), in(2_F, X)} = 0.
:- #count {X : vrs(X), out(1_C, X), in(2_F, X)} = 0.


In [63]:
print("\n".join(gen_rules("1_D", "2_D", ["="])))

:- #count {A : vrs(A), in(1_D, A), out(2_D, A)} = 0,
   #count {B : vrs(B), in(1_D, B), in(2_D, B)} > 0,
   #count {C : vrs(C), out(1_D, C), in(2_D, C)} > 0.
:- #count {A : vrs(A), in(1_D, A), out(2_D, A)} > 0,
   #count {B : vrs(B), in(1_D, B), in(2_D, B)} > 0,
   #count {C : vrs(C), out(1_D, C), in(2_D, C)} > 0.
:- #count {A : vrs(A), in(1_D, A), out(2_D, A)} > 0,
   #count {B : vrs(B), in(1_D, B), in(2_D, B)} = 0,
   #count {C : vrs(C), out(1_D, C), in(2_D, C)} > 0.
:- #count {A : vrs(A), in(1_D, A), out(2_D, A)} > 0,
   #count {B : vrs(B), in(1_D, B), in(2_D, B)} > 0,
   #count {C : vrs(C), out(1_D, C), in(2_D, C)} = 0.
ir(X, r25) :- in(1_D, X), out(2_D, X).
:- #count {X : vrs(X), in(1_D, X), in(2_D, X)} = 0.
ir(X, r25) :- out(1_D, X), in(2_D, X).


In [64]:
print("\n".join(gen_rules("1_E", "2_G", ["o", "="])))

:- #count {A : vrs(A), in(1_E, A), out(2_G, A)} = 0,
   #count {B : vrs(B), in(1_E, B), in(2_G, B)} > 0,
   #count {C : vrs(C), out(1_E, C), in(2_G, C)} > 0.
:- #count {A : vrs(A), in(1_E, A), out(2_G, A)} > 0,
   #count {B : vrs(B), in(1_E, B), in(2_G, B)} = 0,
   #count {C : vrs(C), out(1_E, C), in(2_G, C)} > 0.
:- #count {A : vrs(A), in(1_E, A), out(2_G, A)} > 0,
   #count {B : vrs(B), in(1_E, B), in(2_G, B)} > 0,
   #count {C : vrs(C), out(1_E, C), in(2_G, C)} = 0.
vr(X, r26) ; ir(X, r26) :- in(1_E, X), out(2_G, X).
:- #count {X : vrs(X), in(1_E, X), in(2_G, X)} = 0.
vr(X, r26) ; ir(X, r26) :- out(1_E, X), in(2_G, X).


In [65]:
# MIR and Decoding rules (Standard)

In [75]:
def decoding_rules():
    
    rel_list = list(all_rels)
    mir_rules = [] ##
    for i in range(len(rel_list)):
        for j in range(i+1, len(rel_list)):
            mir_rules.append(':- rel(X, Y, "{}"), rel(X, Y, "{}"), concept2(X, N1), concept2(X, N2).'.format(rel_list[i], rel_list[j]))
    
    t = []
    for rel in rel_list:
        t.append('not rel(X, Y, "{}")'.format(rel))
    t.append('concept2(X, N1)')
    t.append('concept2(Y, N2)')
    t.append('N1 < N2')
    t.append('not ncf(X)')
    t.append('not ncf(Y)')
    at_least_one_rule = ':- {}.'.format(", ".join(t)) ##
    
    rel_def = [] ##
    for rel in rel_list:
        t_ = []
        for i in range(3):
            t_.append('{1}hint(X, Y, {0})'.format(i, "" if sym_dict[rel][i] else "not "))
        rel_def.append('rel(X, Y, "{}") :- {}.'.format(rel, ", ".join(t_)))
    
    ncf_rules = [] ##
    for i in range(3):
        ncf_rules.append('hint(X, Y, {}) :- concept2(X, N1), concept2(Y, N2), N1 < N2, vrs(R), {}(X, R), {}(Y, R), not ncf(X), not ncf(Y).'.format(i, idx_rel_map[i][0], idx_rel_map[i][1]))
    
    all_rules = mir_rules
    all_rules.append(at_least_one_rule)
    all_rules.extend(rel_def)
    all_rules.extend(ncf_rules)
    return all_rules

In [76]:
print("\n".join(decoding_rules()))

:- rel(X, Y, "<"), rel(X, Y, "!"), concept2(X, N1), concept2(X, N2).
:- rel(X, Y, "<"), rel(X, Y, "o"), concept2(X, N1), concept2(X, N2).
:- rel(X, Y, "<"), rel(X, Y, "="), concept2(X, N1), concept2(X, N2).
:- rel(X, Y, "<"), rel(X, Y, ">"), concept2(X, N1), concept2(X, N2).
:- rel(X, Y, "!"), rel(X, Y, "o"), concept2(X, N1), concept2(X, N2).
:- rel(X, Y, "!"), rel(X, Y, "="), concept2(X, N1), concept2(X, N2).
:- rel(X, Y, "!"), rel(X, Y, ">"), concept2(X, N1), concept2(X, N2).
:- rel(X, Y, "o"), rel(X, Y, "="), concept2(X, N1), concept2(X, N2).
:- rel(X, Y, "o"), rel(X, Y, ">"), concept2(X, N1), concept2(X, N2).
:- rel(X, Y, "="), rel(X, Y, ">"), concept2(X, N1), concept2(X, N2).
:- not rel(X, Y, "<"), not rel(X, Y, "!"), not rel(X, Y, "o"), not rel(X, Y, "="), not rel(X, Y, ">"), concept2(X, N1), concept2(Y, N2), N1 < N2, not ncf(X), not ncf(Y).
rel(X, Y, "<") :- not hint(X, Y, 0), hint(X, Y, 1), hint(X, Y, 2).
rel(X, Y, "!") :- hint(X, Y, 0), not hint(X, Y, 1), hint(X, Y, 2).
rel(X,

In [70]:
import pickle as pkl
import anytree

In [38]:
anytree_ = None
with open('Temp_Pickle_Data/cen_test/anytree.pkl', 'rb') as f:
    anytree_ = pkl.load(f)

In [39]:
anytree_.keys()

dict_keys(['CEN', 'NDC'])

In [40]:
anytree_['CEN']['CEN'].children[0].children[0].children[0].name
#root.name
#print(anytree.RenderTree(node=root))

'"CEN_P1"'

In [41]:
sibling_disjointness_rules = []
coverage_rules = []
concept_rules = []
isa_rules = []

for i, tax_name in enumerate(anytree_.keys()):
    root = anytree_[tax_name][tax_name].children[0]
    all_rules = gen_tax_rules(root, tax_id=i)
    sibling_disjointness_rules.extend(all_rules[0])
    coverage_rules.extend(all_rules[1])
    concept_rules.extend(all_rules[2])
    isa_rules.extend(all_rules[3])

In [42]:
print("\n".join(sibling_disjointness_rules))

ir(X, r0) :- in("CEN_Northeast", X), in("CEN_Midwest", X).
:- #count {X : vrs(X), in("CEN_Northeast", X), out("CEN_Midwest", X)} = 0.
:- #count {X : vrs(X), in("CEN_Midwest", X), out("CEN_Northeast", X)} = 0.
ir(X, r0) :- in("CEN_Northeast", X), in("CEN_South", X).
:- #count {X : vrs(X), in("CEN_Northeast", X), out("CEN_South", X)} = 0.
:- #count {X : vrs(X), in("CEN_South", X), out("CEN_Northeast", X)} = 0.
ir(X, r0) :- in("CEN_Northeast", X), in("CEN_West", X).
:- #count {X : vrs(X), in("CEN_Northeast", X), out("CEN_West", X)} = 0.
:- #count {X : vrs(X), in("CEN_West", X), out("CEN_Northeast", X)} = 0.
ir(X, r0) :- in("CEN_Midwest", X), in("CEN_South", X).
:- #count {X : vrs(X), in("CEN_Midwest", X), out("CEN_South", X)} = 0.
:- #count {X : vrs(X), in("CEN_South", X), out("CEN_Midwest", X)} = 0.
ir(X, r0) :- in("CEN_Midwest", X), in("CEN_West", X).
:- #count {X : vrs(X), in("CEN_Midwest", X), out("CEN_West", X)} = 0.
:- #count {X : vrs(X), in("CEN_West", X), out("CEN_Midwest", X)} = 

In [43]:
print("\n".join(coverage_rules))

out("CEN_USA", X) :- out("CEN_Northeast", X), out("CEN_Midwest", X), out("CEN_South", X), out("CEN_West", X).
out("CEN_Northeast", X) :- out("CEN_P1", X), out("CEN_P2", X).
out("CEN_Midwest", X) :- out("CEN_B1", X), out("CEN_B2", X), out("CEN_B3", X).
out("NDC_USA", X) :- out("NDC_Midwest", X), out("NDC_Northeast", X), out("NDC_Southeast", X), out("NDC_Southwest", X), out("NDC_West", X).
out("NDC_West", X) :- out("NDC_K1", X).
out("NDC_K1", X) :- out("NDC_A1", X), out("NDC_A2", X), out("NDC_A3", X).


In [44]:
print("\n".join(concept_rules))

concept2("CEN_USA", 0).
concept2("CEN_Northeast", 0).
concept("CEN_P1", 0, 0).
concept("CEN_P2", 0, 1).
concept2("CEN_Midwest", 0).
concept("CEN_B1", 0, 2).
concept("CEN_B2", 0, 3).
concept("CEN_B3", 0, 4).
concept("CEN_South", 0, 5).
concept("CEN_West", 0, 6).
concept2("NDC_USA", 1).
concept("NDC_Midwest", 1, 0).
concept("NDC_Northeast", 1, 1).
concept("NDC_Southeast", 1, 2).
concept("NDC_Southwest", 1, 3).
concept2("NDC_West", 1).
concept2("NDC_K1", 1).
concept("NDC_A1", 1, 4).
concept("NDC_A2", 1, 5).
concept("NDC_A3", 1, 6).


In [45]:
print("\n".join(isa_rules))

ir(X, r0) :- in("CEN_Northeast", X), out("CEN_USA", X).
:- #count {X : vrs(X), in("CEN_Northeast", X), in("CEN_USA", X)} = 0.
ir(X, r0) :- in("CEN_Midwest", X), out("CEN_USA", X).
:- #count {X : vrs(X), in("CEN_Midwest", X), in("CEN_USA", X)} = 0.
ir(X, r0) :- in("CEN_South", X), out("CEN_USA", X).
:- #count {X : vrs(X), in("CEN_South", X), in("CEN_USA", X)} = 0.
ir(X, r0) :- in("CEN_West", X), out("CEN_USA", X).
:- #count {X : vrs(X), in("CEN_West", X), in("CEN_USA", X)} = 0.
ir(X, r0) :- in("CEN_P1", X), out("CEN_Northeast", X).
:- #count {X : vrs(X), in("CEN_P1", X), in("CEN_Northeast", X)} = 0.
ir(X, r0) :- in("CEN_P2", X), out("CEN_Northeast", X).
:- #count {X : vrs(X), in("CEN_P2", X), in("CEN_Northeast", X)} = 0.
ir(X, r0) :- in("CEN_B1", X), out("CEN_Midwest", X).
:- #count {X : vrs(X), in("CEN_B1", X), in("CEN_Midwest", X)} = 0.
ir(X, r0) :- in("CEN_B2", X), out("CEN_Midwest", X).
:- #count {X : vrs(X), in("CEN_B2", X), in("CEN_Midwest", X)} = 0.
ir(X, r0) :- in("CEN_B3", X), 

In [22]:
def gen_tax_rules(root, tax_id=0, concept_count=0):
    children = root.children
    sd_r = []
    cov_r = []
    concept_r = []
    isa_r = []
    if len(children) > 0:
        sd_r = [gen_sibling_disjointness(children[n1].name, children[n2].name, 0) for n1 in range(len(children)) for n2 in range(n1+1, len(children))]
        cov_r = [gen_coverage_rule(root.name, list(map(lambda x: x.name, children)))]
        concept_r = [gen_concept2_rule(root.name, tax_id)]
        isa_r = [gen_isa_rule(child.name, root.name, 0) for child in children]
        for child in children:
            t_sd_r, t_cov_r, t_concept_r, t_isa_r, concept_count = gen_tax_rules(child, tax_id, concept_count)
            sd_r.extend(t_sd_r)
            cov_r.extend(t_cov_r)
            concept_r.extend(t_concept_r)
            isa_r.extend(t_isa_r)
    else:
        concept_r = [gen_concept_rule(root.name, tax_id, concept_count)]
        concept_count += 1
    return sd_r, cov_r, concept_r, isa_r, concept_count

In [18]:
def gen_coverage_rule(parent: str, children: list):  # For every non-leaf node
    lhs = 'out({}, X)'.format(parent)
    rhs = ", ".join(list(map(lambda x: 'out({}, X)'.format(x), children)))
    return "{} :- {}.".format(lhs, rhs)

In [19]:
def gen_concept2_rule(node: str, tax_num: int):
    return "concept2({}, {}).".format(node, tax_num)
def gen_concept_rule(node: str, tax_num: int, concept_num: int):
    return "concept({}, {}, {}).".format(node, tax_num, concept_num)

In [20]:
def gen_sibling_disjointness(n1: str, n2: str, idx: int):  # For every pair of siblings
    r1 = ir_helper(n1=n1, n2=n2, rel1="in", rel2="in", idx=idx)
    r2 = ":- {}.".format(not_filter_helper(n1=n1, n2=n2, rel1="in", rel2="out", sign="=", r_var="X"))
    r3 = ":- {}.".format(not_filter_helper(n1=n2, n2=n1, rel1="in", rel2="out", sign="=", r_var="X"))
    return "\n".join([r1,r2,r3])

In [21]:
def gen_isa_rule(child: str, parent: str, idx: int):  # For every parent-child relation
    r1 = ir_helper(n1=child, n2=parent, rel1="in", rel2="out", idx=idx)
    r2 = ":- {}.".format(not_filter_helper(n1=child, n2=parent, rel1="in", rel2="in", sign="=", r_var="X"))
    return "\n".join([r1,r2])

In [46]:
articulations = None
with open('Temp_Pickle_Data/cen_test/taxDesc.pkl', 'rb') as f:
    articulations = pkl.load(f)
print(articulations)

              Node1 Relation            Node2
0         "CEN_USA"   parent  "CEN_Northeast"
1         "CEN_USA"   parent    "CEN_Midwest"
2         "CEN_USA"   parent      "CEN_South"
3         "CEN_USA"   parent       "CEN_West"
4   "CEN_Northeast"   parent         "CEN_P1"
5   "CEN_Northeast"   parent         "CEN_P2"
6     "CEN_Midwest"   parent         "CEN_B1"
7     "CEN_Midwest"   parent         "CEN_B2"
8     "CEN_Midwest"   parent         "CEN_B3"
9         "NDC_USA"   parent    "NDC_Midwest"
10        "NDC_USA"   parent  "NDC_Northeast"
11        "NDC_USA"   parent  "NDC_Southeast"
12        "NDC_USA"   parent  "NDC_Southwest"
13        "NDC_USA"   parent       "NDC_West"
14       "NDC_West"   parent         "NDC_K1"
15         "NDC_K1"   parent         "NDC_A1"
16         "NDC_K1"   parent         "NDC_A2"
17         "NDC_K1"   parent         "NDC_A3"
18        "CEN_USA"        =        "NDC_USA"
19       "CEN_West"        o       "NDC_West"
20       "CEN_West"        !  "NDC

In [82]:
articulation_rules = []
for idx, row in articulations.iterrows():
    n1 = row['Node1']
    rel = row['Relation'].split(",")
    n2 = row['Node2']
    if rel[0] != 'parent':
        articulation_rules.extend(gen_rules(n1, n2, rel))
print("\n".join(articulation_rules))

:- #count {A : vrs(A), in("CEN_USA", A), out("NDC_USA", A)} = 0,
   #count {B : vrs(B), in("CEN_USA", B), in("NDC_USA", B)} > 0,
   #count {C : vrs(C), out("CEN_USA", C), in("NDC_USA", C)} > 0.
:- #count {A : vrs(A), in("CEN_USA", A), out("NDC_USA", A)} > 0,
   #count {B : vrs(B), in("CEN_USA", B), in("NDC_USA", B)} > 0,
   #count {C : vrs(C), out("CEN_USA", C), in("NDC_USA", C)} > 0.
:- #count {A : vrs(A), in("CEN_USA", A), out("NDC_USA", A)} > 0,
   #count {B : vrs(B), in("CEN_USA", B), in("NDC_USA", B)} = 0,
   #count {C : vrs(C), out("CEN_USA", C), in("NDC_USA", C)} > 0.
:- #count {A : vrs(A), in("CEN_USA", A), out("NDC_USA", A)} > 0,
   #count {B : vrs(B), in("CEN_USA", B), in("NDC_USA", B)} > 0,
   #count {C : vrs(C), out("CEN_USA", C), in("NDC_USA", C)} = 0.
ir(X, r42) :- in("CEN_USA", X), out("NDC_USA", X).
:- #count {X : vrs(X), in("CEN_USA", X), in("NDC_USA", X)} = 0.
ir(X, r42) :- out("CEN_USA", X), in("NDC_USA", X).
:- #count {A : vrs(A), in("CEN_West", A), out("NDC_West", 